<a href="https://colab.research.google.com/github/DhruvMenon2003/Kaboose-App-Face-Recognition-System-for-User-Verification-Built-on-Homomorphic-Encryption/blob/main/Kaboose_Verification_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
#Imported Light Partial Homomoprhic Encryption Library(LightPHE)
#Building multiple cryptosystems based on RSA, ElGamal, Paillier and Goldwasser-Micali PHE algorithms
cs_RSA=LightPHE(algorithm_name="RSA")
cs_ElGamal=LightPHE(algorithm_name="ElGamal")
cs_Paillier=LightPHE(algorithm_name="Paillier")
cs_GoldwasserMicali=LightPHE(algorithm_name="Goldwasser-Micali")
#Test RSA to learn what is Multiplicatively Homomoprhic
m1=17
m2=23
c1=cs_RSA.encrypt(plaintext=m1)
c2=cs_RSA.encrypt(plaintext=m2)
assert cs_RSA.decrypt(c1*c2)==m1*m2
#Test Paillier to learn what is Additively Homomorphic + Multiplication by a scalar + Regeneration of CipherText
c1=cs_Paillier.encrypt(plaintext=m1)
c2=cs_Paillier.encrypt(plaintext=m2)
assert cs_Paillier.decrypt(c1+c2)==m1+m2
c1_new= cs_Paillier.regenerate_ciphertext(ciphertext=c1)
assert cs_Paillier.decrypt(c1_new)==m1
#Test Goldwasser-Micali what is  Exclusively Homomorphic
c1=cs_GoldwasserMicali.encrypt(plaintext=m1)
c2=cs_GoldwasserMicali.encrypt(plaintext=m2)
assert cs_GoldwasserMicali.decrypt(c1^c2)==m1^m2

### Kaboose App Face Recognition System for User Verification Built on Homomorphic Encryption
### What is the idea of homomorphic encryption?
Perform any operation on encrypted data without decrypting.
### Encryption Algorithm used : **CKKS**
CKKS has been an essential tool in enabling privacy-preserving computations, especially in fields like data analysis, machine learning, and medical research, where the need to perform computations on sensitive data is common. It offers a way to do this without compromising the privacy of the underlying data.
### How it works?
Finds Euclidean Distance between encrypted image embeddings . If comparison is successful(no matches with existing data) it goes to the DeepFace AI model for verification after which it is deleted. If comparison is unsuccessful(matches with existing data), the image is directly deleted.
### Features
* The Cloud Database provides the resources for the computationally intensive
operations like Euclidean Distance calculations
* Protects sensitive data in cloud against cyberattacks by adding noise to during encryption.








In [54]:
#Import Required Libraries
import tenseal as ts
from deepface import DeepFace
import base64
import math
#Link to app
#Only allow 1 user to upload 1 image. Allow user to visualize uploaded image by opening "Page1.jpg" or the page with the image icon centered inside a mobile icon with "This is me" below. Grab the path and upload to Google CoLab.
#We are going to find vector representations of uploaded facial images.
#Finding Embeddings: (Data is right now on Google CoLab Client)
img1_path = (from user1)
img2_path = (from user2)
....
imgn_path = (from usern)
img1_embedding = DeepFace.represent(model_name="Facenet",img_path=img1_path)
img2_embedding = DeepFace.represent(model_name="Facenet",img_path=img2_path)
imgn_embedding = DeepFace.represent(model_name="Facenet",img_path=imgn_path)
#Write code for a try catch loop to catch the error.If "No face" error appears, then open "Page3.jpg" or page asking user to retry
#Initialize Private key-Public Key pairs : (Data is right now on Google CoLab Client)
context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree=8192, coeff_mod_bit_sizes = [60,40,40,60] )
context.generate_galois_keys()
context.global_scale = 2**40
secret_context = context.serialize(save_secret_key=True)
write_data("secret.txt", secret_context)
context.make_context_public()
public_context = context.serialize()
write_data("public.txt", public_context)
del context,secret_context,public_context #very important step for security!!
#Code to Transfer Public.txt to the chosen Docker/Cloud Client

#Encryption: (Data will be encrypted on Google CoLab Client)
context = ts.context_from(read_data("secret.txt"))
enc_v1 = ts.ckks_vector(context, img1_embedding)
enc_v2 = ts.ckks_vector(context, img2_embedding)
...
enc_vn = ts.ckks_vector(context, imgn_embedding)
#Write Code to Transfer Encrypted embeddings to the chosen Docker/Cloud client (Data is right now Cloud Client)
#Store the uploaded embeddings on the chosen Docker/cloud client
write_data("enc_v1.txt", enc_v1.serialize())
write_data("enc_v2.txt", enc_v2.serialize())
...
write_data("enc_vn.txt", enc_vn.serialize())
#Code to Transfer files to the Cloud
del context,enc_v1,enc_v2,..enc_vn  #very important step for security!!

#Comparison step: Each embedding with the other embeddings and the embeddings that are already stored on the Cloud Client from previous users.(Encrypted Data is now on Cloud Client)
context = ts.context_from(read_data("public.txt"))
enc_v1 = ts.lazy_ckks_tensor_from(read_data("enc_v1.txt"))
enc_v2 = ts.lazy_ckks_tensor_from(read_data("enc_v2.txt"))
...
enc_vn = ts.lazy_ckks_tensor_from(read_data("enc_vn.txt"))
enc_v1.link_context(context)
enc_v2.link_context(context)
...
enc_vn.link_context(context)
#Take difference all possible pairs of vector embeddings and take the dot product with itself.
#For eg.
euclidean_squared = enc_v1-enc_v2
euclidean_squared = euclidean_squared.dot(euclidean_squared)
euclidean_squared = enc_v2-enc_v3
euclidean_squared = euclidean_squared.dot(euclidean_squared)
...
euclidean_squared = enc_v1-enc_vn
euclidean_squared = euclidean_squared.dot(euclidean_squared)
del context,enc_v1,enc_v2, euclidean_squared #very important step for security!!
#Each embedding will have a vector of euclidean_squared distance values from other embeddings.
write_data(file_name="euclidean_squared.txt",file_context=euclidean_squared.serialize())
#Write code to transfer "euclidean_squared.txt" from Cloud client to the Google CoLab Client
#Decryption: (Data in file "euclidean_squared.txt" will be decrypted on Google CoLab Client)
....
#Data is right now in Google CoLab Client
context = ts.context_from(read_data("secret.txt"))
euclidean_squared = ts.lazy_ckks_tensor_from(read_data("euclidean_squared.txt"))
euclidean_squared.link_context(context)
#Execute in a Loop for each embedding.
for embedding enc_v1,
for i=2:n ( Each embedding is compared with n-1 other embeddings)
  euclidean distance = math.sqrt(euclidean_squared[i].decrypt()[0])
  if euclidean_distance < 10:
    #Open "Page3.jpg" or page asking user to retry with the retry button
    #Write code to Delete image embedding stored in the .txt file in the cloud client
#Move to "Page6.jpg" or page with the video icon
DeepFace.stream(db_path=img1_path,model_name="Facenet")
#Render the video on webcam in place of the video icon centered inside the mobile icon (refer to "Page7.jpg")
#If the output from the Model = True, then the user is verified so move to "Page2.jpg" or the page with the video icon and back button
#If the output from the Model = False, then the user is not verified so Open "Page3.jpg" or page asking user to retry with the retry button

del context #very important step for security!!
del "secret.txt" #We don't want anyone to decrypt the sensitive data stored in the cloud! Most Important Step for Security, Do not forget!
Delete the image files and video taken by DeepFace, uploaded to GoogleCoLab by ending session! (The encrypted embeddings are safely stored in cloud client)
def write_data(file_name, data):

    if type(data) == bytes:
        #bytes to base64
        data = base64.b64encode(data)

    with open(file_name, 'wb') as f:
        f.write(data)

def read_data(file_name):
    with open(file_name, "rb") as f:
        data = f.read()

    #base64 to bytes
    return base64.b64decode(data)






SyntaxError: invalid syntax (<ipython-input-54-c4283a1fcc63>, line 5)

**bold text**